In [6]:
from IPython import parallel
engines = parallel.Client()
engines.block = True
print(engines.ids)

[0, 1, 2, 3]


In [8]:
engines [0].execute('a = 2')
engines [0].execute('b = 10')
engines [0].execute('c = a + b')

<AsyncResult: execute:finished>

In [9]:
engines[0].pull('c')

12

In [11]:
# Parallelization

engines[0].execute('a = 2')
engines[0].execute('b = 10')
engines[1].execute('a = 9')
engines[1].execute('b = 7')
engines[0:2].execute('c = a + b')

<AsyncResult: execute:finished>

In [14]:
engines [0:2]. pull('c')

[12, 16]

In [16]:
dview2 = engines[0:2]
dview = engines.direct_view()

In [22]:
import random
A0 = [[random.random() for e in range(3)] for e in range(3)]
A1 = [[random.random() for e in range(3)] for e in range(3)]
B0 = [[random.random() for e in range(3)] for e in range(3)]
B1 = [[random.random() for e in range(3)] for e in range(3)]

In [23]:
# haciendo todo a manopla

dview2.execute('import numpy as np')
engines [0]. push(dict(A=A0, B=B0))
engines [1]. push(dict(A=A1, B=B1))
dview2.execute('C = np.dot(A,B)')
dview2.pull('C')

[array([[1.18624521, 0.73341464, 0.81530649],
        [0.76392366, 0.45323444, 0.49769691],
        [0.82361954, 0.54723302, 0.75634098]]),
 array([[0.69615155, 0.93910892, 0.47539599],
        [1.03117888, 1.54323854, 1.09030936],
        [0.42437927, 0.62654507, 0.29887277]])]

In [26]:
# definiendo la funcion y mandandola con apply
# apply hace directamente el push, execute y pull

def mul(A, B):
    import numpy as np
    C = np.dot(A, B)
    return C
C = engines [0].apply(mul , A0 , B0)
C

array([[1.18624521, 0.73341464, 0.81530649],
       [0.76392366, 0.45323444, 0.49769691],
       [0.82361954, 0.54723302, 0.75634098]])

In [28]:
# mapeando para los distintos motores

[C0 , C1] = dview2.map(mul ,[A0 , A1],[B0 , B1])
[C0 , C1]

[array([[1.18624521, 0.73341464, 0.81530649],
        [0.76392366, 0.45323444, 0.49769691],
        [0.82361954, 0.54723302, 0.75634098]]),
 array([[0.69615155, 0.93910892, 0.47539599],
        [1.03117888, 1.54323854, 1.09030936],
        [0.42437927, 0.62654507, 0.29887277]])]

In [29]:
# Load-Balanced View of Engines

engines.block = True # we use the blocking mode since it simplifies the code.
lview2 = engines.load_balanced_view(targets = [0, 1])
lview = engines.load_balanced_view()